## Anomaly detection and pattern extraction with Spark, Cassandra and Scala

Today, geo-located data is available in a number of domains, ranging from healthcare to financial markets, to social services. In all these domains, extracting patterns and detecting anomalies and novelties from data has very concrete business outcomes. 

Anomaly detection can be defined as the process of finding which samples in the given dataset do not follow the given patterns and behave as though they were produced by a different mechanism. From detection follows action. Depending on the domain and the use case, we define them as anomalies or novelties and these signals are the triggers for applications such as personalized marketing and fraud alerting and notification.

As more data gets ingested/produced via digital services, it’s key to perform this sort of analytics at scale. In the open source space, technologies such as Spark and Cassandra are definitely instrumental to implement and execute modern data pipelines at scale.

This Oriole is divided in two parts: Firstly, I will show how to collect data from Cassandra and bring it up to Spark for further analysis. In the second part of this Oriole, I will explore a number of techniques for detecting anomalies based on three different techniques:

  - Statistics and Histograms
  - Process Mining and Graph Analytics
  - Clustering for Geo-Located Data with DBSCAN

For this analysis, we are going to use the Gowalla Dataset [1]. The Gowalla dataset consists of a table of events, registered by anonymized users. Each event registers a user checking into a geolocated venue at a specific timestamp. The dataset is available at https://snap.stanford.edu/data/loc-gowalla.html

[1] E. Cho, S. A. Myers, J. Leskovec. Friendship and Mobility: Friendship and Mobility: User Movement in Location-Based Social Networks ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (KDD), 2011.



#### Setup

This notebook is running scala code and interfaces to a Spark cluster using the [Apache Toree](https://toree.incubator.apache.org/) project. Furthermore, Spark reads the data from Cassandra tables. Spark interfaces to Cassandra via the [Cassandra-Spark connector](https://github.com/datastax/spark-cassandra-connector). 

At the time of compiling this notebook, Spark 1.6.1 and Cassandra 3.5 were used. Here below the command to install the Spark - Scala Kernel on Jupiter. More instructions on this topic are available on Apache Toree [website](https://toree.incubator.apache.org/) and [github pages](https://github.com/apache/incubator-toree).

```
sudo jupyter-toree install --spark_home=${SPARK_HOME} 
--spark_opts='--packages com.datastax.spark:spark-cassandra
-connector_2.10:1.6.0,graphframes:graphframes:0.1.0-spark1.6 
--conf spark.cassandra.connection.host=localhost --conf 
spark.executor.memory=4g --conf spark.driver.memory=4g'
```

In [1]:
// Scala version
sc.version

1.6.1

### Connecting to Cassandra

In [2]:
//sql context
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.functions._

val sqlContext  = new SQLContext(sc)
import sqlContext.implicits._

In [3]:
// spark-cassandra connector
import com.datastax.spark.connector._
import com.datastax.spark.connector.cql._

import org.apache.spark.sql.cassandra.CassandraSQLContext
val cc = new CassandraSQLContext(sc)

#### SQL queries in Cassandra

Cassandra is exposed via a SQL context, so there is not need to learn a separate syntax as Spark will map the query to the available features of the underlying storage system. See below a simple query accessing the name and the id of venues from a cassandra table. Also remember that sql statements are _staged_ but not _executed_ until some actual [actions](http://spark.apache.org/docs/latest/programming-guide.html#actions) needs to be computed. Examples of actions are for instance, **count**(), **first**(), **collect**().

In [4]:
val venues   = cc.sql("select vid, name from lbsn.venues").as("venues")

In [5]:
venues.count()

30366

In [6]:
venues.first()

[1073929,Sputnik Gallery]

Sometimes you cannot push to cassandra the full query, as the full query cannot be mapped on the available database functions supported on that specific data store. For instance, Cassandra cannot easily execute joins. In this case, Spark will partition and plan the query _pushing down_ what can be done in Cassandra and perform in Spark the rest of the query. 

More information can be found on Cassandra Documentation about [using Spark SQL to query data](http://docs.datastax.com/en/datastax_enterprise/5.0/datastax_enterprise/spark/sparkSqlOverview.html) or on the [Cassandra Spark Connector](https://github.com/datastax/spark-cassandra-connector) pages.

For example, the following query filters out only those events which were registered in the New York area. As filtering in cassandra cannot by done on columns which are not indexes, this specific query will first move the data form Cassandra to Spark, and then will perform the filtering in Spark. In general, it's a good practice to push down and filter as much data as early as possible. This practice keeps the throughput low and minimize the data transfered from one system to the other.

In [7]:
val events   = cc.sql("""select ts, uid, lat, lon, vid from lbsn.events where
                            lon>-74.2589 and lon<-73.7004 and 
                            lat> 40.4774 and lat< 40.9176
                      """).as("events")

In [8]:
events.count()

138948

In [9]:
events.first()

[2009-12-17 14:58:23.0,83942,40.7586191119,-73.9888966084,225371]

Before diving into anomaly detection of geo-located data, let's perform some more basic queries. Herebelow, it is shown how to count events registered by user uid=0, and how to retrieve the name of venue id 7239827. Finally, the third query prints out the first five rows of the venue table, in no particular order.

In [14]:
// User 0: how many checkins?
events.where("uid=0").count()

27

In [15]:
venues.where("vid = 7239827").first()

[7239827,Central Park Manhattan ]

In [16]:
venues.show(5, false)

+-------+--------------------+
|vid    |name                |
+-------+--------------------+
|1073929|Sputnik Gallery     |
|1350193|Kaori's Closet Tokyo|
|1425555|Club Spain          |
|7160165|La Quinta - Brooklyn|
|285750 |La Sorrentina       |
+-------+--------------------+
only showing top 5 rows



#### Joining Cassandra tables in Spark.

One of the advantages of connecting Cassandra and Spark, is the fact that you can now merge and join Cassandra tables.

In [21]:
val df_ny = events.
  join(venues, events("events.vid") === venues("venues.vid"), "inner").
  select("ts", "uid", "lat", "lon", "events.vid","venues.name")

df_ny.show(5,false)

+---------------------+------+-------------+--------------+-------+--------------------+
|ts                   |uid   |lat          |lon           |vid    |name                |
+---------------------+------+-------------+--------------+-------+--------------------+
|2009-12-17 14:58:23.0|83942 |40.7586191119|-73.9888966084|225371 |Smith's Bar & Grill |
|2010-08-18 03:08:36.0|166583|40.697517133 |-74.175481333 |1276055|Jake's Coffeehouse  |
|2010-09-14 14:15:22.0|41568 |40.7538465143|-73.9845085144|1308099|The Southwest Porch |
|2009-12-04 16:24:03.0|74122 |40.721437025 |-73.9978015423|15998  |La Esquina          |
|2009-12-04 16:23:01.0|74122 |40.7211204821|-73.9982306578|32674  |Blip.tv Headquarters|
+---------------------+------+-------------+--------------+-------+--------------------+
only showing top 5 rows



The spark table joined above is going to be the starting point for our anomaly analysis.    
Each row records the event's timestamp, the user id, the geo-location (latitude and longitude) of venue and finally the venue id and name.

#### Executing SQL statements as code.

Spark dataframes can also be filtered and transformed programmatically via a number of [pre-defined functions](https://spark.apache.org/docs/1.6.1/api/scala/#org.apache.spark.sql.functions$), such as min, sum, stddev, and many more. Some of those are shown in the next code sections. 

Next to the default set of pre-defined dataframe and column functions, it is possible to define the user-defined-functions (udf's). In the code below, we will create two UDF's to transform the timestamp to the day of the week and the hour of the day values, computed according to a given local timezone.

In [22]:
// UDF functions for SQL-like operations on columns
import org.joda.time.DateTime
import org.joda.time.DateTimeZone

import java.sql.Timestamp
import org.apache.spark.sql.functions.udf

val  dayofweek = udf( (ts: Timestamp, tz: String) => {
  val dt = new DateTime(ts,DateTimeZone.forID(tz))
  // sunday starts at 0
  dt.getDayOfWeek() - 1
})

val  localhour = udf( (ts: Timestamp, tz: String) => {
  val dt = new DateTime(ts,DateTimeZone.forID(tz))
  dt.getHourOfDay()
})

In [23]:
val newyork_tz = "America/New_York"

val df = df_ny.
  withColumn("dow",  dayofweek($"ts", lit(newyork_tz))).
  withColumn("hour", localhour($"ts", lit(newyork_tz))).
  as("events").cache()
  
df.show(5, false)


+---------------------+------+-------------+--------------+-------+--------------------+---+----+
|ts                   |uid   |lat          |lon           |vid    |name                |dow|hour|
+---------------------+------+-------------+--------------+-------+--------------------+---+----+
|2009-12-17 14:58:23.0|83942 |40.7586191119|-73.9888966084|225371 |Smith's Bar & Grill |3  |17  |
|2010-08-18 03:08:36.0|166583|40.697517133 |-74.175481333 |1276055|Jake's Coffeehouse  |2  |6   |
|2010-09-14 14:15:22.0|41568 |40.7538465143|-73.9845085144|1308099|The Southwest Porch |1  |17  |
|2009-12-04 16:24:03.0|74122 |40.721437025 |-73.9978015423|15998  |La Esquina          |4  |19  |
|2009-12-04 16:23:01.0|74122 |40.7211204821|-73.9982306578|32674  |Blip.tv Headquarters|4  |19  |
+---------------------+------+-------------+--------------+-------+--------------------+---+----+
only showing top 5 rows



## Anomaly Detection

### Histogram based
#### Basic statistics in Spark

The following code section shows how to collect global statistics and histograms per hour of the day and per day of the week. Histograms can be made more specific by aggregating the events according to a number of factors, such as:

 - venue
 - geographical area
 - popular users
 - 1st, 2nd friend's circle
 
If you are interested, in multiple slicing and dicing option, Spark as a [cube function](https://spark.apache.org/docs/1.5.1/api/scala/index.html#org.apache.spark.sql.DataFrame) as well.  
To start, let's compute the histogram, accumulating all events and aggregating by hour of the day and by day of the week.  


In [25]:
// histogram day of the week events
df.groupBy($"dow").count().show()

+---+-----+
|dow|count|
+---+-----+
|  0|14640|
|  1|15385|
|  2|15686|
|  3|16179|
|  4|16558|
|  5|18085|
|  6|15849|
+---+-----+



In [26]:
// histogram hour of the day events
df.groupBy($"hour").count().show(24,false)

+----+-----+
|hour|count|
+----+-----+
|0   |2681 |
|1   |1272 |
|2   |852  |
|3   |554  |
|4   |323  |
|5   |550  |
|6   |937  |
|7   |2157 |
|8   |3615 |
|9   |4586 |
|10  |4903 |
|11  |5576 |
|12  |7257 |
|13  |8142 |
|14  |7687 |
|15  |7254 |
|16  |7519 |
|17  |7495 |
|18  |8621 |
|19  |8970 |
|20  |7167 |
|21  |6127 |
|22  |4747 |
|23  |3390 |
+----+-----+



#### Statistics in Spark: venue-specific histograms

Moving on, let's have a look on how to create an specific histogram for each venue.   
In this case, we will store the histogram as a vector. First, let's convert the day-of-the-week to a vector.

In [27]:
import breeze.linalg._
import breeze.linalg.DenseVector

import org.apache.spark.mllib.linalg.{Vector,Vectors}

In [28]:
// vector histogram: the RDD way

def toVector(i: Int, length:Int) = {
  DenseVector((0 to length-1).map(x => if (x == i) 1.0 else 0.0).toArray)
}

In [29]:
df.select($"vid", $"dow").map(r => (r.getLong(0),toVector(r.getInt(1), 7))).first()

(225371,DenseVector(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0))

#### Pair RDDs: reduce by Key

We will now **reduceByKey** those weekly and daily vectors by applying vectors arithmetics. In this way, we can collect the probability of an event happening at a specific day of the week for each venue in the dataset. This is a much more detailed analysis since different venues, such as restaurants, musea and train stations have different daily and weekly histogram patterns.

In [30]:
val dow_hist = df.
  select($"vid", $"dow").
  map(r => (r.getLong(0),toVector(r.getInt(1), 7))).
  reduceByKey(_ + _).
  mapValues(x => Vectors.dense((x / sum(x)).toArray)).
  toDF("vid", "dow_hist")

  

In [31]:
dow_hist.show(3, false)

+-------+-------------------------------+
|vid    |dow_hist                       |
+-------+-------------------------------+
|713995 |[0.25,0.0,0.0,0.0,0.5,0.25,0.0]|
|4415530|[0.0,0.0,0.0,0.0,0.0,1.0,0.0]  |
|486265 |[0.25,0.0,0.0,0.0,0.0,0.5,0.25]|
+-------+-------------------------------+
only showing top 3 rows



#### Scoring events according to venues histograms

In [32]:
val df_probs = df.
  join(dow_hist, df("vid") === dow_hist("vid"), "inner").
  select("ts", "uid", "lat", "lon", "events.vid", "dow", "dow_hist").
  cache()

df_probs.show(5,false)

+---------------------+------+-------------+--------------+-------+---+-------------------------------+
|ts                   |uid   |lat          |lon           |vid    |dow|dow_hist                       |
+---------------------+------+-------------+--------------+-------+---+-------------------------------+
|2010-03-15 04:45:53.0|155999|40.7131656401|-74.0353098807|713995 |0  |[0.25,0.0,0.0,0.0,0.5,0.25,0.0]|
|2010-06-19 13:08:21.0|41235 |40.7131656401|-74.0353098807|713995 |5  |[0.25,0.0,0.0,0.0,0.5,0.25,0.0]|
|2010-04-23 20:47:34.0|10687 |40.7131656401|-74.0353098807|713995 |4  |[0.25,0.0,0.0,0.0,0.5,0.25,0.0]|
|2010-10-08 04:53:51.0|74222 |40.7131656401|-74.0353098807|713995 |4  |[0.25,0.0,0.0,0.0,0.5,0.25,0.0]|
|2010-09-25 10:02:38.0|181811|40.76603735  |-73.78952235  |4415530|5  |[0.0,0.0,0.0,0.0,0.0,1.0,0.0]  |
+---------------------+------+-------------+--------------+-------+---+-------------------------------+
only showing top 5 rows



#### Histogram based anomaly detection
We will now score the probability of a given event based on the histograms we have just computed here above. For that, we are going to craft a new UDF which will select a given element of a vector based on the value provided by a different column. 

In [33]:
val  nth = udf( (i:Int, arr: Vector) => {
  arr.toArray.lift(i).getOrElse(0.0)
})

df_probs.select($"ts", $"uid", $"vid", $"dow", nth($"dow", $"dow_hist").as("dow_prob")).show(3,false)

+---------------------+------+------+---+--------+
|ts                   |uid   |vid   |dow|dow_prob|
+---------------------+------+------+---+--------+
|2010-03-15 04:45:53.0|155999|713995|0  |0.25    |
|2010-06-19 13:08:21.0|41235 |713995|5  |0.25    |
|2010-04-23 20:47:34.0|10687 |713995|4  |0.5     |
+---------------------+------+------+---+--------+
only showing top 3 rows



#### Histograms for anomaly detection: putting it all together

Let's repeat the same exercise for the histograms binned by hour of the day. And finally, let's merge and compute the probability of each given event, given the venue, the hour of the day, and the day of the week. Events with lower probabilities are less likely to happen.

In [34]:
// same for hour of the day

val hour_hist = df.
  select($"vid", $"hour").
  map(r => (r.getLong(0),toVector(r.getInt(1), 24))).
  reduceByKey(_ + _).
  mapValues(x => Vectors.dense((x / sum(x)).toArray)).
  toDF("vid", "hour_hist")

hour_hist.show(3, false)

+-------+---------------------------------------------------------------------------------------------------+
|vid    |hour_hist                                                                                          |
+-------+---------------------------------------------------------------------------------------------------+
|713995 |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25]|
|4415530|[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]  |
|486265 |[0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.0,0.0]|
+-------+---------------------------------------------------------------------------------------------------+
only showing top 3 rows



In [35]:
val df_probs = df.
  join(dow_hist, df("vid") === dow_hist("vid"), "inner").
  join(hour_hist, df("vid") === hour_hist("vid"), "inner").
  select( 
    $"ts", 
    $"uid", 
    $"lat", 
    $"lon", 
    $"events.vid", 
    nth($"hour", $"hour_hist").as("hour_prob"), 
    nth($"dow",  $"dow_hist").as("dow_prob")).
  cache()

df_probs.show(5,false)

+---------------------+------+-------------+--------------+-----+--------------------+-------------------+
|ts                   |uid   |lat          |lon           |vid  |hour_prob           |dow_prob           |
+---------------------+------+-------------+--------------+-----+--------------------+-------------------+
|2010-08-13 23:28:29.0|125327|40.7490532543|-73.9680397511|11831|0.015873015873015872|0.19047619047619047|
|2010-09-29 12:08:36.0|578   |40.7490532543|-73.9680397511|11831|0.12698412698412698 |0.2222222222222222 |
|2010-09-05 13:20:27.0|578   |40.7490532543|-73.9680397511|11831|0.047619047619047616|0.12698412698412698|
|2010-07-21 07:02:07.0|578   |40.7490532543|-73.9680397511|11831|0.07936507936507936 |0.2222222222222222 |
|2010-07-20 14:24:06.0|578   |40.7490532543|-73.9680397511|11831|0.07936507936507936 |0.14285714285714285|
+---------------------+------+-------------+--------------+-----+--------------------+-------------------+
only showing top 5 rows



### Process mining based

The first step, in order to do process mining, is to collect sequences of events. In particular, the following code will take chronologically consecutive events and bundle them in pairs for a specific user. These pairs consists of two venue ids, namely source and destination, defining where each user is coming from, going to respectively. 

The steps in the following code are:

  - Convert the DataFrame to an RDD
  - Select uid as the key for the PairRDD
  - Reshape the PairRDD from "tall" to "wide"
  - Sort chronologically all the checked-in venues for each user
  - Extract pairs from each sequence of checked-in venues per user
  - Reshape the PairRDD from "wide" to "tall" again
  - Convert back the PairRDD to a DataFrame

In [36]:
// process mining
val g_df = events.
  select($"ts", $"uid", $"vid").
  rdd.
  map(row => (row.getLong(1), List( (row.getTimestamp(0), row.getLong(2)) ))).
  reduceByKey(_ ++ _).
  mapValues( x =>
    x.sortWith(_._1.getTime < _._1.getTime).
      map(_._2)
  ).
  mapValues(_.sliding(2).toList).
  flatMap(_._2).
  map(
    _ match {
      case List(a, b) => Some((a, b))
      case _ => None
  }).
  flatMap(x => x).
  toDF("src", "dst").
  cache()

This newly created DataFrame is used to create a graph, where the nodes are the venues and the edges are connections of users checking-in from the one venue to the next.

In [1]:
g_df.show(5)

+------+------+
|   src|   dst|
+------+------+
|255148|603177|
|603177|603177|
|603177|603177|
|603177|603177|
|603177|188022|
+------+------+
only showing top 5 rows



In [14]:
val edges_df = g_df.
  groupBy($"src",$"dst").
  count().
  select($"src",$"dst").
  filter($"src" !== $"dst").
  cache()

edges_df.show(5)

+------+------+
|   src|   dst|
+------+------+
|886164|487279|
| 12149|748034|
|104364| 15079|
|248066|853505|
|655145|985182|
+------+------+
only showing top 5 rows



In [15]:
val nodes_df = edges_df.
  select($"src").
  unionAll(edges_df.select($"dst")).
  distinct().
  toDF("id").
  cache()

nodes_df.show(5)

+------+
|    id|
+------+
|220031|
|105831|
| 28031|
| 11831|
|434031|
+------+
only showing top 5 rows



In [16]:
println(s"# nodes = ${nodes_df.count()}")
println(s"# edges = ${edges_df.count()}")

# nodes = 21429
# edges = 108757


#### Graph and Page Rank

The above table describes how users are moving from venue to venue. We can now calculate which venues attract more users. This can be done using the page rank algorithm. The PageRank algorithm outputs a probability distribution used to represent the likelihood that a person randomly walking in the city will arrive at any particular venue. This analysis can be executed in Spark using [GraphFrames](http://graphframes.github.io/). GraphFrames is a package for Apache Spark which provides DataFrame-based graph analytics, including the PageRank algorithm.

In [19]:
import org.graphframes.GraphFrame

val g = GraphFrame(nodes_df, edges_df)

In [18]:
val results = g.pageRank.resetProbability(0.05).maxIter(5).run()

In [66]:
val vertices = results.vertices.select("id", "pagerank")
val popular_venues = vertices.join(venues, vertices("id") === venues("vid"), "inner").select("vid", "pagerank", "name")

popular_venues.sort($"pagerank".desc).show(13, false)

+------+------------------+---------------------------------+
|vid   |pagerank          |name                             |
+------+------------------+---------------------------------+
|12505 |27.881643763277896|LGA LaGuardia Airport            |
|23261 |26.385424367704715|JFK John F. Kennedy International|
|11844 |22.11151212984308 |Times Square                     |
|13022 |17.736644176952165|Grand Central Terminal           |
|24963 |16.479439108529313|EWR Newark Liberty International |
|11875 |10.202291677899952|Madison Square Garden            |
|12525 |10.166984788526063|The Museum of Modern Art (MoMA)  |
|11720 |10.0143847403669  |Yankee Stadium                   |
|106840|9.76488986972341  |Union Square                     |
|11834 |8.960647198372016 |Bryant Park                      |
|12313 |8.15588197657451  |Empire State Building            |
|14151 |6.999067399261288 |Rockefeller Center               |
|17710 |6.986291334077661 |Madison Square Park              |
+------+

As shown above, this algorithm provides a "popularity" factor for each checked-in venue. This feature can be used to further discriminate anomalies based on the rank of the venue, for instance combining it with the probability of checking in a specific time of the day.

### Geo-Location: density based

We will now cluster events based on the [DBSCAN algorithm](https://en.wikipedia.org/wiki/DBSCAN). DBSCAN is clustering events depending on the density of the events provided. Since the clusters emerge locally by looking for neighboring points, clusters of various shapes can be detected. Points that are isolated and too far from any other point are assigned to a special cluster of outliers. These discerning properties make the DBSCAN algorithm a good candidate for clustering geolocated events.

In [67]:
%addjar https://github.com/natalinobusa/nak/raw/master/nak_2.10-1.3.jar

Starting download from https://github.com/natalinobusa/nak/raw/master/nak_2.10-1.3.jar
Finished download of nak_2.10-1.3.jar


Let's prepare the data by transforming the events DataFrame, into a PairRDD. In particular, for geolocated data, we choose the key to be the user identifier, and the value to be the aggregated list of all check-ins posted by that given user. The geolocated data is arranged in a n-by-2 matrix, where the first column represents the latitude and the second column the longitude. 

In [20]:
val e_df = events.
  select("uid","lat","lon").
  rdd.map(row => (row.getLong(0), Array(row.getDouble(1), row.getDouble(2))) ).
  reduceByKey( _ ++ _).
  mapValues(v => new DenseMatrix(v.length/2,2,v, 0, 2, true))


In [21]:
def formatUserEvents(x: Tuple2[Long, DenseMatrix[Double]]) : Unit = {
    val arr = x._2
    val n = math.min( 5 , arr.rows) - 1
    val slice = arr(0 to n, ::)
    println(s"uid = ${x._1}")
    println(s"events count = ${arr.rows}")
    println("lat,lon = ")
    println(slice)
    if (arr.rows > 5) println(s"... ${arr.rows- 5} more rows")
    println("-"*30)
}

See below a formatted output describing the events related to three users:

In [22]:
e_df.take(3).foreach(e => formatUserEvents(e))

uid = 33590
events count = 355
lat,lon = 
40.735486752   -73.979792352   
40.73723724    -73.980824105   
40.7424578176  -73.9837482386  
40.70736247    -74.008827565   
40.737524907   -73.978869923   
... 350 more rows
------------------------------
uid = 139605
events count = 4
lat,lon = 
40.7126315333  -74.0444852833  
40.6993813387  -74.0393972397  
40.69856285    -74.03974055    
40.6987988833  -74.0400195     
------------------------------
uid = 108050
events count = 1
lat,lon = 
40.7425115937  -74.0060305595  
------------------------------


We will now cluster the events for each user according to the DBSCAN algorithm. This algorithm with cluster those user's events in groups. The rest of the code below reduces those groups to bounding boxes. Next we will use the extracted bounding boxes to score events.

In [40]:
import breeze.numerics._
import breeze.linalg._

def euclideanDistance (a: DenseVector[Double], b: DenseVector[Double]) = norm(a-b, 2)

// 1deg at 40deg latitude is 111034.61 meters
// set radius at about 200 mt (0.002 * 111034.61)
// which is 0.002 in decimal degrees https://en.wikipedia.org/wiki/Decimal_degrees

val eps = 0.002
val min_points = 3

In [41]:
import nak.cluster._
import nak.cluster.GDBSCAN._

def dbscan(v : breeze.linalg.DenseMatrix[Double]) = {

  val gdbscan = new GDBSCAN(
    DBSCAN.getNeighbours(eps, distance=euclideanDistance),
    DBSCAN.isCorePoint(min_points)
  )

  // core DBSCAN algorithm
  val clusters = gdbscan cluster v
  
  // reducing the clusters to bounding boxes
  // for simplicity: each user could 
  clusters.map(
    cluster => (
      cluster.id.toInt, 
      cluster.points.size, 
      cluster.points.map(_.value(0)).min,
      cluster.points.map(_.value(1)).min,
      cluster.points.map(_.value(0)).max,
      cluster.points.map(_.value(1)).max
    )
  )
}

In [42]:
val bboxRdd = e_df.mapValues(dbscan(_)).cache()

Let's convert back the RDDs to a DataFrame. Now we have a table describing clusters. Each row defines a cluster in terms of user id, cluster id, the number of cluster's events and the bounding box of the cluster. Each user can have multiple clusters, and some users might have no cluster at all.

In [43]:
val bbox_df = bboxRdd.
  flatMapValues(x => x).
  map(x => (x._1, x._2._1, x._2._2,x._2._3,x._2._4,x._2._5,x._2._6)).
  toDF("uid", "cid", "csize", "lat_min", "lon_min", "lat_max", "lon_max").
  filter($"cid" > 0)

bbox_df.show(10)

+-----+---+-----+-------------+--------------+-------------+--------------+
|  uid|cid|csize|      lat_min|       lon_min|      lat_max|       lon_max|
+-----+---+-----+-------------+--------------+-------------+--------------+
|33590|  1|   12|  40.73723724| -73.980824105| 40.737524907| -73.978869923|
|33590|  2|    7|40.7419666616| -73.984329364| 40.742535639| -73.982899202|
|33590|  3|    6| 40.705680584| -74.009272538|  40.70736247| -74.008827565|
|33590|  5|   70|40.7356195403|-73.9754120581| 40.736689817| -73.973834661|
|33590|  7|  102|40.7021876333| -74.011605514|  40.70539715| -74.009128417|
|33590| 14|   13| 40.737480983| -73.974163929| 40.737480983| -73.974163929|
|33590| 20|   18|  40.73773491| -73.983837394| 40.739129691|  -73.98285782|
|33590| 21|   15|40.7371600561|-73.9878805558|   40.7379291|-73.9848974977|
|33590| 26|   14| 40.744767148|   -73.9770325|    40.744856| -73.975315143|
|33590| 44|    8|40.7355633167|-73.9912224008|40.7377660167|-73.9884488333|
+-----+---+-

#### Scoring Events: looking for outliers

We will now score events, and look if some of them are located outside the computed clusters' bounding boxes. Firstly, we join the table of events with the table of clusters. Let's filter out users which do not have enough points as those users have no clusters associated with them and there is no sufficient data to determine outliers. In the code above, we need a user to have at least 3 events in a region of 0.1 degrees in order to have a DBSCAN cluster.

In [45]:
val bbox_events = events.
  join(bbox_df, events("events.uid") === bbox_df("uid"), "full").
  select("events.ts","events.uid","lat","lon","lat_min","lon_min","lat_max","lon_max").
  filter($"lat_min".isNotNull)

bbox_events.show(5,false)

+---------------------+----+-------------+--------------+-------------+--------------+-------------+--------------+
|ts                   |uid |lat          |lon           |lat_min      |lon_min       |lat_max      |lon_max       |
+---------------------+----+-------------+--------------+-------------+--------------+-------------+--------------+
|2009-12-06 22:47:17.0|4831|40.7489512339|-73.9955051586|40.7489512339|-73.9955051586|40.750613794 |-73.993434906 |
|2009-12-06 22:47:17.0|4831|40.7489512339|-73.9955051586|40.7503537066|-73.99460925  |40.7527147753|-73.9925895262|
|2009-12-06 22:47:17.0|4831|40.7489512339|-73.9955051586|40.7482404588|-73.9927482605|40.7503009825|-73.9919929265|
|2009-12-06 22:47:03.0|4831|40.7492524   |-73.9951317833|40.7489512339|-73.9955051586|40.750613794 |-73.993434906 |
|2009-12-06 22:47:03.0|4831|40.7492524   |-73.9951317833|40.7503537066|-73.99460925  |40.7527147753|-73.9925895262|
+---------------------+----+-------------+--------------+-------------+-

In [177]:
case class EventBbox(
  ts: Timestamp,
  uid: Long, 
  lat:Double, 
  lon: Double, 
  lat_min:Double, 
  lon_min:Double, 
  lat_max:Double, 
  lon_max:Double)
  
case class EventDetected(
  ts: Timestamp,
  uid: Long, 
  lat: Double, 
  lon: Double, 
  bbox: Boolean
)

In [178]:
def bbox_check( x:EventBbox): Boolean = {
  x.lon >= x.lon_min &
  x.lon <= x.lon_max &
  x.lat >= x.lat_min &
  x.lat <= x.lat_max   
}

The following code uses the newer Dataset API, which is a DataFrame where row are handled as typed objects. In particular, we are converting the events row into a `EventDetected` object and then we check if the event is within the boundary of the given cluster. Since each user might have more than one cluster, we check each event against all the user's clusters and we consider it an outlier if none of the check is returns a positive outcome. 

In [49]:
val scored_events = bbox_events.
  as[EventBbox].
  map(x => EventDetected(x.ts, x.uid, x.lat, x.lon, bbox_check(x))).
  groupBy($"ts", $"uid").
  reduce( (x,y) => if (x.bbox) x else y ).
  map(x => x._2).
  cache()

Here below the outlier scoring for `uid=4831`. 

In [50]:
scored_events.
  filter(_.uid==4831).
  show(10)

+--------------------+----+-------------+--------------+-----+
|                  ts| uid|          lat|           lon| bbox|
+--------------------+----+-------------+--------------+-----+
|2009-11-16 12:55:...|4831|40.7515061333|-73.9928534833| true|
|2009-12-02 13:32:...|4831|40.7504689667|   -73.9987564|false|
|2009-12-06 22:44:...|4831|   40.7515816|  -73.99429768| true|
|2009-12-06 22:47:...|4831|40.7489512339|-73.9955051586| true|
|2009-11-16 12:50:...|4831|40.7524687739|-73.9944867037| true|
|2009-11-16 12:54:...|4831|40.7525623527|-73.9944184053| true|
|2009-11-16 12:37:...|4831|40.7529089392|-73.9966748478|false|
|2009-11-16 12:59:...|4831|40.7473494768|-74.0008061373|false|
|2009-12-02 13:32:...|4831|  40.75168975|-73.9941437667| true|
|2009-11-16 12:34:...|4831|   40.7518756|  -73.99460925| true|
+--------------------+----+-------------+--------------+-----+
only showing top 10 rows



As you can see three items are found outside those bounding boxes. Although this is not yet a strong indicator for an anomaly per se, it can constitute a very relevant signal if combined with other signals as seen above. Many improvements can be done to the above core idea, for instance, by including relations and interaction between users and more refined analysis of clusters, using for instance convex hulls instead of bounding boxes and so forth.

I hope you enjoyed this notebook, thanks for keeping up with me till here. Best wishes for your data science projects!